# Estimate First-Level Models

*Written by Luke Chang

This notebook contains code used to estimate first level models for a number of the datasets.

- HCP
    - Includes code to download from HCP AWS S3 Bucket
    - Gambling
    - Social
    - WM
    - Language
- MIT MID 
    - Collected by Co-Author Livia Tomova
    - Accessed via https://openneuro.org/datasets/ds003242/versions/1.0.0
    - Includes GLM code
- Vanderbilt MID
    - Shared by Co-Authors Greg Samanez-Larkin, Jaime Castrellon, & David Zald
- Rutgers Social Gambling
    - Shared by Co-Authors Dominic Fareri & Maurcio Delgado
- Rutgers Trust Game
    - Shared by Co-Authors Dominic Fareri & Maurcio Delgado
- Neurosynth Gene Expression
    - Shared by Co-Author Luke Chang
    - Available on https://neurosynth.org/)
- Vanderbilt PET
    - Shared by Co-Authors Greg Samanez-Larkin, Jaime Castrellon, & David Zald
- Sweet Taste
    - Accessed via https://openneuro.org/datasets/ds000229/versions/00001
    - Includes GLM Code
- Milkshake Data 
    - Shared by Eric Stice and Sonja Yokum
- UCLA BART
    - Accessed via https://openneuro.org/datasets/ds000030/versions/1.0.0
    - Includes GLM Code
- Self Referential
    - Shared by Co-Author Andy Chen
- Boulder Pain
    - Shared by Co-Author Luke Chang
    - Available at https://neurovault.org/collections/504/
- Pittsburgh IAPS
    - Shared by Co-Author Luke Chang
    - Available at https://neurovault.org/collections/1964/
- Stop Signal Task
    - Accessed via https://neurovault.org/collections/1807/
- MIT Deprivation
    - Collected by Co-Author Livia Tomova
    - Accessed via https://openneuro.org/datasets/ds003242/versions/1.0.0
    - Includes GLM Code
- Mixed Gamble Task
    - Accessed via https://openneuro.org/datasets/ds000005/versions/00001
    - Includes GLM Code
- Naturalistic Viewing
    - Shared by Co-Author Luke Chang
    - Available at https://openneuro.org/datasets/ds003521/versions/1.0.0
- Facial Expressions
    - Shared by Co-Author Luke Chang
    - Available at https://osf.io/f9gyd/

In [ ]:
%matplotlib inline

import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from copy import deepcopy
from nltools.data import Brain_Data, Design_Matrix
from nltools.mask import expand_mask, roi_to_brain
from nltools.stats import zscore, regress, find_spikes
from nltools.file_reader import onsets_to_dm
from nltools.external import glover_hrf
from nilearn.plotting import view_img, glass_brain, plot_stat_map
from tqdm import tqdm
import nibabel as nib

import warnings
warnings.filterwarnings('ignore')

def make_motion_covariates(mc, tr=2.0):
    '''Create motion covariates regressors from realignment parameters
    
    Args:
        mc: (pd.DataFrame) realignment parameters
        tr: (float) repetition time
        
    Returns:
        design_matrix: (Design_Matrix) instance

    '''
    
    z_mc = zscore(mc)
    all_mc = pd.concat([z_mc, z_mc**2, z_mc.diff(), z_mc.diff()**2], axis=1)
    all_mc.fillna(value=0, inplace=True)
    return Design_Matrix(all_mc, sampling_freq=1/tr)




# Download and Clean HCP Data 
Data is downloaded from OpenAccess AWS S3

In [ ]:
base_dir = '/Storage/Data/HCP'

meta = pd.read_csv(os.path.join(base_dir, 'unrestricted_ljchang_5_16_2021_18_44_14.csv'))
sub_list = list(meta['Subject'].values)

tasks = ['GAMBLING','LANGUAGE','SOCIAL','WM']

for sub in tqdm(sub_list):
    for task in tasks:
        if not os.path.exists(os.path.join(base_dir, 'Data', str(sub))):
            os.mkdir(os.path.join(base_dir, 'Data', str(sub)))
        if not os.path.exists(os.path.join(base_dir, 'Data', str(sub), task)):
            os.mkdir(os.path.join(base_dir, 'Data', str(sub), task))
        if task == 'WM':
            !aws s3 cp s3://hcp-openaccess/HCP/{sub}/MNINonLinear/Results/tfMRI_{task}/tfMRI_{task}_hp200_s4_level2vol.feat/cope9.feat/stats/pe1.nii.gz /Storage/Data/HCP/Data/{sub}/{task}/cope9/
            !aws s3 cp s3://hcp-openaccess/HCP/{sub}/MNINonLinear/Results/tfMRI_{task}/tfMRI_{task}_hp200_s4_level2vol.feat/cope10.feat/stats/pe1.nii.gz /Storage/Data/HCP/Data/{sub}/{task}/cope10/      
        else:
            !aws s3 cp s3://hcp-openaccess/HCP/{sub}/MNINonLinear/Results/tfMRI_{task}/tfMRI_{task}_hp200_s4_level2vol.feat/cope1.feat/stats/pe1.nii.gz /Storage/Data/HCP/Data/{sub}/{task}/cope1/
            !aws s3 cp s3://hcp-openaccess/HCP/{sub}/MNINonLinear/Results/tfMRI_{task}/tfMRI_{task}_hp200_s4_level2vol.feat/cope2.feat/stats/pe1.nii.gz /Storage/Data/HCP/Data/{sub}/{task}/cope2/
    

## Clean Gambling

In [ ]:
task = 'GAMBLING'

sub_list = list(set([x.split('/')[5] for x in file_list_1]) & set([x.split('/')[5] for x in file_list_2]))
sub_list.sort()
file_list_1 = []; file_list_2 = [];
for sub in sub_list:
    file_list_1.append(os.path.join(base_dir, 'Data', sub, task, 'cope1','pe1.nii.gz'))
    file_list_2.append(os.path.join(base_dir, 'Data', sub, task, 'cope2','pe1.nii.gz'))
    
print(len(file_list_1), len(file_list_2))
punish = Brain_Data(file_list_1)
reward = Brain_Data(file_list_2)

# Write out as hdf5
punish.write(os.path.join(base_dir, 'Analyses',task, f'HCP_{task}_Punish_n{len(punish)}.hdf5'))
reward.write(os.path.join(base_dir, 'Analyses',task, f'HCP_{task}_Reward_n{len(punish)}.hdf5'))

## Clean Social

In [ ]:
task = 'SOCIAL'

sub_list = list(set([x.split('/')[5] for x in glob.glob(os.path.join(base_dir, 'Data','*', task, 'cope1','pe1.nii.gz'))]) & set([x.split('/')[5] for x in glob.glob(os.path.join(base_dir, 'Data','*', task, 'cope2','pe1.nii.gz'))]))
sub_list.sort()
file_list_1 = []; file_list_2 = [];
for sub in sub_list:
    file_list_1.append(os.path.join(base_dir, 'Data', sub, task, 'cope1','pe1.nii.gz'))
    file_list_2.append(os.path.join(base_dir, 'Data', sub, task, 'cope2','pe1.nii.gz'))

print(len(file_list_1), len(file_list_2))
random = Brain_Data(file_list_1)
tom = Brain_Data(file_list_2)

# Write out as hdf5
random.write(os.path.join(base_dir, 'Analyses', f'HCP_{task}', f'HCP_{task}_Random_n{len(random)}.hdf5'))
tom.write(os.path.join(base_dir, 'Analyses', f'HCP_{task}', f'HCP_{task}_TOM_n{len(tom)}.hdf5'))

## Clean Language

In [ ]:
task = 'LANGUAGE'

sub_list = list(set([x.split('/')[5] for x in glob.glob(os.path.join(base_dir, 'Data','*', task, 'cope1','pe1.nii.gz'))]) & set([x.split('/')[5] for x in glob.glob(os.path.join(base_dir, 'Data','*', task, 'cope2','pe1.nii.gz'))]))
sub_list.sort()
file_list_1 = []; file_list_2 = [];
for sub in sub_list:
    file_list_1.append(os.path.join(base_dir, 'Data', sub, task, 'cope1','pe1.nii.gz'))
    file_list_2.append(os.path.join(base_dir, 'Data', sub, task, 'cope2','pe1.nii.gz'))

print(len(file_list_1), len(file_list_2))
math = Brain_Data(file_list_1)
story = Brain_Data(file_list_2)

# Write out as hdf5
math.write(os.path.join(base_dir, 'Analyses', f'HCP_{task}', f'HCP_{task}_Math_n{len(random)}.hdf5'))
story.write(os.path.join(base_dir, 'Analyses', f'HCP_{task}', f'HCP_{task}_Story_n{len(tom)}.hdf5'))


## Clean Working Memory

In [ ]:
task = 'WM'

sub_list = list(set([x.split('/')[5] for x in glob.glob(os.path.join(base_dir, 'Data','*', task, 'cope9','pe1.nii.gz'))]) & set([x.split('/')[5] for x in glob.glob(os.path.join(base_dir, 'Data','*', task, 'cope10','pe1.nii.gz'))]))
sub_list.sort()
file_list_1 = []; file_list_2 = [];
for sub in sub_list:
    file_list_1.append(os.path.join(base_dir, 'Data', sub, task, 'cope9','pe1.nii.gz'))
    file_list_2.append(os.path.join(base_dir, 'Data', sub, task, 'cope10','pe1.nii.gz'))

print(len(file_list_1), len(file_list_2))
wm_2bk = Brain_Data(file_list_1)
wm_0bk = Brain_Data(file_list_2)

# Write out as hdf5
wm_2bk.write(os.path.join(base_dir, 'Analyses', f'HCP_{task}', f'HCP_{task}_2Back_n{len(random)}.hdf5'))
wm_0bk.write(os.path.join(base_dir, 'Analyses', f'HCP_{task}', f'HCP_{task}_0Back_n{len(tom)}.hdf5'))


# MIT MID Dataset

In [ ]:
def load_bids_events(subject='01', session='b', run=1, tr=2.0, task='midloc'):
    '''Create a design_matrix instance from BIDS event file
    
    Args:
        subject: (str) subject id
        session: (str) scanning session
        run: (int) run number
        tr: (float) repetition time

    Returns:
        design_matrix: (Design_Matrix) instance
    
    '''

    n_tr = nib.load(os.path.join(beta_dir, 'derivatives','fmriprep',f'sub-SAXSISO{subject}{session}','func',f'sub-SAXSISO{subject}{session}_task-{task}_run-00{run}_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz')).shape[-1]
    events = pd.read_csv(os.path.join(beta_dir, f'sub-SAXSISO{subject}{session}','func',f'sub-SAXSISO{subject}{session}_task-{task}_run-00{run}_events.tsv'), sep='\t')
    events = events[['onset','duration','trial_type']]
    events.columns = ['Onset', 'Duration', 'Stim']
    events = events.loc[~events['Stim'].isnull()]
    return onsets_to_dm(events, sampling_freq=1/tr, run_length=n_tr)

beta_dir = '/Storage/Data/social_deprivation/'


subject = '01'
session = 'b'
run = 1
tr = 2.0

# for subject in tqdm([1,2,3,4,8,9,10,11,12,13,14,15,17,18,19,21,22,24,26,27,28,30,32,33,34,35,36,38,39,40,41,42]):
for subject in tqdm([1,2,3,4,8,10,11,12,13,14,15,17,18,19,21,24,26,27,28,30,32,33,34,35,36,38,39,41,42]):

    # Load data
    data = Brain_Data(os.path.join(beta_dir, 'derivatives','fmriprep',f'sub-SAXSISO{subject:02}{session}','func',f'sub-SAXSISO{subject:02}{session}_task-midloc_run-00{run}_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz'))

    # Smooth 
    data = data.smooth(fwhm=6)

    # Rescale Data so that each voxel is scaled proportional to percent signal change
    data = data.scale()
    
    # confound covars
    confounds = pd.read_csv(os.path.join(beta_dir, 'derivatives','fmriprep',f'sub-SAXSISO{subject:02}{session}','func',f'sub-SAXSISO{subject:02}{session}_task-midloc_run-00{run}_desc-confounds_regressors.tsv'), sep='\t')

    # motion covars
    mc = confounds[['trans_x','trans_y','trans_z','rot_x', 'rot_y', 'rot_z']]
    mc_cov = make_motion_covariates(mc, tr)

    # events
    onsets = load_bids_events(subject=f'{subject:02}', session=session, run=run, tr=tr)

    # spikes
    spikes = data.find_spikes(global_spike_cutoff=3, diff_spike_cutoff=3)

    # Convolve, add linear drift, dct, intercept
    dm_cov = onsets.convolve().add_dct_basis(duration=120).add_poly(order=1, include_lower=True)

    # add spikes
    dm_cov = dm_cov.append(mc_cov, axis=1).append(Design_Matrix(spikes.iloc[:, 1:], sampling_freq=1/tr), axis=1)

    #regress
    data.X = dm_cov
    stats = data.regress()

    write out betas
    reward = stats['beta'][np.where(data.X.columns == 'Rewarded_c0')[0]]
    reward.write(os.path.join(beta_dir, 'derivatives','L1_Betas_Rescaled', f'sub-SAXSISO{subject:02}b_task-midloc_Reward_beta_smooth6mm.nii.gz'))

    noreward = stats['beta'][np.where(data.X.columns == 'NonRewarded_c0')[0]]
    noreward.write(os.path.join(beta_dir, 'derivatives','L1_Betas_Rescaled', f'sub-SAXSISO{subject:02}b_task-midloc_NoReward_beta_smooth6mm.nii.gz'))



# MIT Social Deprivation Dataset

In [ ]:


tr = 2.0
task = 'ls'

for subject in tqdm([1,2,3,4,8,9,10,11,12,13,14,15,17,18,19,21,22,24,26,27,28,30,32,33,34,35,36,38,39,40,41,42]):
    for session in ['b','f','s']:
        for run in range(1,7):
            try:
                # Load data
                data = Brain_Data(os.path.join(beta_dir, 'derivatives','fmriprep',f'sub-SAXSISO{subject:02}{session}','func',f'sub-SAXSISO{subject:02}{session}_task-{task}_run-00{run}_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz'))

                # Smooth 
                data = data.smooth(fwhm=6)

                # Rescale Data so that each voxel is scaled proportional to percent signal change
                data = data.scale()

                # confound covars
                confounds = pd.read_csv(os.path.join(beta_dir, 'derivatives','fmriprep',f'sub-SAXSISO{subject:02}{session}','func',f'sub-SAXSISO{subject:02}{session}_task-{task}_run-00{run}_desc-confounds_regressors.tsv'), sep='\t')

                # motion covars
                mc = confounds[['trans_x','trans_y','trans_z','rot_x', 'rot_y', 'rot_z']]
                mc_cov = make_motion_covariates(mc, tr)

                # events - NOTE: load bids events function does not work for other datasets not TOMOVA
                onsets = load_bids_events(subject=f'{subject:02}', session=session, run=run, tr=tr, task=task)

                # spikes
                spikes = data.find_spikes(global_spike_cutoff=3, diff_spike_cutoff=3)

                # Convolve, add linear drift, dct, intercept
                dm_cov = onsets.convolve().add_dct_basis(duration=120).add_poly(order=1, include_lower=True)

                # add spikes
                dm_cov = dm_cov.append(mc_cov, axis=1).append(Design_Matrix(spikes.iloc[:, 1:], sampling_freq=1/tr), axis=1)

                #regress
                data.X = dm_cov
                stats = data.regress()

                # Create Condition Average Contrasts
                food_contrast = (stats['beta'][np.array([x.split('_')[0] for x in data.X.columns]) == 'Food']).mean()
                social_contrast = (stats['beta'][np.array([x.split('_')[0] for x in data.X.columns]) == 'Social']).mean()
                control_contrast = (stats['beta'][np.array([x.split('_')[0] for x in data.X.columns]) == 'Control']).mean()

                food_contrast.write(os.path.join(beta_dir, 'derivatives','L1_Betas_Rescaled', f'sub-SAXSISO{subject:02}{session}_task-{task}_run-00{run}_Food_beta_smooth6mm.nii.gz'))
                social_contrast.write(os.path.join(beta_dir, 'derivatives','L1_Betas_Rescaled', f'sub-SAXSISO{subject:02}{session}_task-{task}_run-00{run}_Social_beta_smooth6mm.nii.gz'))
                control_contrast.write(os.path.join(beta_dir, 'derivatives','L1_Betas_Rescaled', f'sub-SAXSISO{subject:02}{session}_task-{task}_run-00{run}_Control_beta_smooth6mm.nii.gz'))
            except:
                print(f'{subject}_{session}_{run} Failed')

## Aggregate Data

In [ ]:
# Average data across runs within subject and drop subjects without full data across sessions
condition = ['Social','Food','Baseline']

cic_sub_list = list(set([os.path.basename(x).split('_')[0] for x in glob.glob(os.path.join(beta_dir, 'derivatives','L1_Betas_Rescaled', '*task-CIC*.nii.gz'))]))
cic_sub_list.sort()

sub_list = list(set([x[:-1] for x in cic_sub_list]))
sub_list.sort()

all_session_data = {}
missing = []
for session in ['s', 'b', 'f']:
    avg_social_beta = {}
    avg_baseline_beta = {}
    avg_food_beta = {}
    for sub in tqdm(sub_list):
        try:
            avg_social_beta[sub] = Brain_Data(glob.glob(os.path.join(beta_dir, 'derivatives', 'L1_Betas_Rescaled', f'{sub}{session}_task-CIC_run-*_Social_beta_smooth6mm.nii.gz'))).mean()
        except:
            avg_social_beta[sub] = np.nan
            f'{sub} - Social Not Available'
            missing.append(sub)
        try:
            avg_baseline_beta[sub] = Brain_Data(glob.glob(os.path.join(beta_dir, 'derivatives', 'L1_Betas_Rescaled', f'{sub}{session}_task-CIC_run-*_Control_beta_smooth6mm.nii.gz'))).mean()
        except:
            avg_baseline_beta[sub] = np.nan
            f'{sub} - Control Not Available'
            missing.append(sub)
        try:
            avg_food_beta[sub] = Brain_Data(glob.glob(os.path.join(beta_dir, 'derivatives', 'L1_Betas_Rescaled', f'{sub}{session}_task-CIC_run-*_Food_beta_smooth6mm.nii.gz'))).mean()
        except:
            avg_food_beta[sub] = np.nan
            f'{sub} - Food Not Available'
            missing.append(sub)
        all_session_data[session] = {'Social':avg_social_beta, 'Baseline':avg_baseline_beta, 'Food':avg_food_beta}
missing = list(set(missing))

# Write out Files to HDF5
for session in ['s', 'b', 'f']:
    for condition in ['Social', 'Baseline', 'Food']:
        Brain_Data([all_session_data[session][condition][sub] for sub in all_session_data[session][condition] if sub not in missing]).write(os.path.join(data_dir, 'Data', 'social_isolation', f'MIT_Deprivation_Session_{session}_Images_{condition}.hdf5'))


# Sweet Taste Dataset

In [ ]:
def load_bids_events(subject=1, task='flavor', run=1, tr=2.0):
    '''Create a design_matrix instance from BIDS event file
    
    Args:
        subject: (str) subject id
        session: (str) scanning session
        run: (int) run number
        tr: (float) repetition time

    Returns:
        design_matrix: (Design_Matrix) instance
    
    '''

    n_tr = nib.load(os.path.join(data_dir, 'derivatives','fmriprep', f'sub-{subject:02}','func',f'sub-{subject:02}_task-{task}_run-{run:02}_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz')).shape[-1]
    events = pd.read_csv(os.path.join(data_dir, f'sub-{subject:02}','func', f'sub-{subject:02}_task-{task}_run-{run:02}_events.tsv'), sep='\t')
    events = events[['onset','duration','stimulus']]
    events.columns = ['Onset', 'Duration', 'Stim']
#     events = events.loc[~events['Stim'] == 'rinse']
    return onsets_to_dm(events, sampling_freq=1/tr, run_length=n_tr)


data_dir = '/Storage/Data/sweet_taste'



tr = 2.0
task = 'flavor'

for subject in tqdm([1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]):
    for run in range(1,4):
        try:

            # Load data
            data = Brain_Data(os.path.join(data_dir, 'derivatives','fmriprep',f'sub-{subject:02}','func',f'sub-{subject:02}_task-{task}_run-{run:02}_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz'))

            # Smooth 
            data = data.smooth(fwhm=6)

            # Rescale Data so that each voxel is scaled proportional to percent signal change
            data = data.scale()

            # confound covars
            confounds = pd.read_csv(os.path.join(data_dir, 'derivatives', 'fmriprep', f'sub-{subject:02}','func',f'sub-{subject:02}_task-{task}_run-{run:02}_desc-confounds_regressors.tsv'), sep='\t')

            # motion covars
            mc = confounds[['trans_x','trans_y','trans_z','rot_x', 'rot_y', 'rot_z']]
            mc_cov = make_motion_covariates(mc, tr)

            # events - NOTE: load bids events function does not work for other datasets not TOMOVA
            onsets = load_bids_events(subject=subject, run=run, tr=tr, task=task)

            # spikes
            spikes = data.find_spikes(global_spike_cutoff=3, diff_spike_cutoff=3)

            # Convolve, add linear drift, dct, intercept
            dm_cov = onsets.convolve().add_dct_basis(duration=120).add_poly(order=1, include_lower=True)

            # add spikes
            dm_cov = dm_cov.append(mc_cov, axis=1).append(Design_Matrix(spikes.iloc[:, 1:], sampling_freq=1/tr), axis=1)

            #regress
            data.X = dm_cov
            stats = data.regress()

            # Write out files
            for i, condition in enumerate(list(data.X.columns[:7])):
                stats['beta'][i].write(os.path.join(data_dir, 'derivatives', 'L1_Betas', f'sub-{subject:02}_task-{task}_run-{run:02}_{condition}.nii.gz'))
        
        except:
            print(f'{subject}_{run} Failed')


## Aggregate Runs

In [ ]:
conditions = list(set([os.path.basename(x).split('_')[3] for x in glob.glob(os.path.join(data_dir, 'derivatives', 'L1_Betas', f'sub-*_task-flavor_run-*_*.nii.gz'))]))

all_data = {x:[] for x in conditions}
for subject in tqdm(range(1,16)):
    for condition in conditions:
        all_data[condition].append(Brain_Data(glob.glob(os.path.join(data_dir, 'derivatives', 'L1_Betas', f'sub-{subject:02}_task-flavor_run-*_{condition}_c0.nii.gz'))).mean())

for condition in list(all_data.keys()):
    Brain_Data(all_data[condition]).write(os.path.join(data_dir, 'derivatives', 'L1_Betas', f'Sweet_Taste_{condition}.hdf5'))

# UCLA BART Dataset

In [ ]:
def load_bids_events(subject=1, task='bart', tr=2.0):
    '''Create a design_matrix instance from BIDS event file
    
    Args:
        subject: (str) subject id
        session: (str) scanning session
        run: (int) run number
        tr: (float) repetition time

    Returns:
        design_matrix: (Design_Matrix) instance
    
    '''

    n_tr = nib.load(os.path.join(data_dir, 'derivatives','fmriprep', subject,'func',f'{subject}_task-{task}_bold_space-MNI152NLin2009cAsym_preproc.nii.gz')).shape[-1]
    events = pd.read_csv(os.path.join(data_dir, subject,'func', f'{subject}_task-{task}_events.tsv'), sep='\t')
    events['stimulus'] = np.nan
    for i,row in events.iterrows():
        if row['action'] == 'EXPLODE':
            condition = 'explode'
        elif row['action'] == 'CASHOUT':
            condition = 'cash'
        elif row['action'] == 'ACCEPT':
            if row['trial_type']=='BALOON':
                condition = 'pumps'
            elif row['trial_type']=='CONTROL':
                condition = 'control_pumps'

        events.loc[i,'stimulus'] = condition
    events.loc[events['stimulus']=='explode', 'duration'] = 1
    events = events[['onset','duration','stimulus']]
    events.dropna(inplace=True)
    events.columns = ['Onset', 'Duration', 'Stim']
    return onsets_to_dm(events, sampling_freq=1/tr, run_length=n_tr)


data_dir = '/Storage/Data/Neuropsychiatric_Phenomics_LA5c/'

metadata = pd.read_csv(os.path.join(data_dir, 'participants.tsv'), sep='\t')
sub_list = list(metadata.query('diagnosis=="CONTROL"')['participant_id'])

task = 'bart'
tr = 2.0
fwhm = 6

missing_subs = []
for subject in tqdm(sub_list[3:]):
    try:
        data = Brain_Data(os.path.join(data_dir, 'derivatives','fmriprep', subject,'func',f'{subject}_task-{task}_bold_space-MNI152NLin2009cAsym_preproc.nii.gz'))

        # smooth
        data = data.smooth(fwhm)

        # confound covars
        confounds = pd.read_csv(os.path.join(data_dir, 'derivatives', 'fmriprep', subject,'func',f'{subject}_task-{task}_bold_confounds.tsv'), sep='\t')

        # motion covars
        mc = confounds[['X','Y','Z','RotX', 'RotY', 'RotZ']]
        mc_cov = make_motion_covariates(mc, tr)

        # events - NOTE: load bids events function does not work for other datasets not TOMOVA
        onsets = load_bids_events(subject=subject, tr=tr, task=task)

        # spikes
        spikes = data.find_spikes(global_spike_cutoff=3, diff_spike_cutoff=3)

        # Convolve, add linear drift, dct, intercept
        dm_cov = onsets.convolve().add_dct_basis(duration=120).add_poly(order=1, include_lower=True)

        # add spikes
        dm_cov = dm_cov.append(mc_cov, axis=1).append(Design_Matrix(spikes.iloc[:, 1:], sampling_freq=1/tr), axis=1)

        #regress
        data.X = dm_cov
        stats = data.regress()

        # Write out files
        for i, condition in enumerate([x for x in data.X.columns if '_c0' in x]):
            stats['beta'][i].write(os.path.join(data_dir, 'derivatives', 'L1_Betas', 'bart', 'smoothed', f'{subject}_task-{task}_{condition[:-3]}.nii.gz'))
    except:
        missing_subs.append(subject)
        print(f'Error with {subject}')

# UCLA Mixed Gambles - Loss Aversion

In [ ]:
def load_bids_events(subject=1, task='mixedgamblestask', run=1, tr=2.0):
    '''Create a design_matrix instance from BIDS event file
    
    Args:
        subject: (str) subject id
        session: (str) scanning session
        run: (int) run number
        tr: (float) repetition time

    Returns:
        design_matrix: (Design_Matrix) instance
    
    '''

    n_tr = nib.load(os.path.join(data_dir, 'derivatives','fmriprep', f'sub-{subject:02}','func',f'sub-{subject:02}_task-{task}_run-{run}_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz')).shape[-1]
    events = pd.read_csv(os.path.join(data_dir, f'sub-{subject:02}','func', f'sub-{subject:02}_task-{task}_run-{run:02}_events.tsv'), sep='\t')
    events['stimulus'] = np.nan
    for i,row in events.iterrows():
        events['stimulus'].iloc[i] = f"Gain{int(row['gain'])}_Loss{int(row['loss'])}_Response{int(row['respnum'])}"

    events = events[['onset','duration','stimulus']]
    events.columns = ['Onset', 'Duration', 'Stim']
    return onsets_to_dm(events, sampling_freq=1/tr, run_length=n_tr)


data_dir = '/Storage/Data/loss_aversion'

tr = 2.0
task = 'mixedgamblestask'

for subject in tqdm(range(1,17)):
    for run in range(1,3):
        data = Brain_Data(os.path.join(data_dir, 'derivatives','fmriprep',f'sub-{subject:02}','func',f'sub-{subject:02}_task-{task}_run-{run}_space-MNI152NLin2009cAsym_desc-preproc_bold.nii.gz'))

        # confound covars
        confounds = pd.read_csv(os.path.join(data_dir, 'derivatives', 'fmriprep', f'sub-{subject:02}','func',f'sub-{subject:02}_task-{task}_run-{run}_desc-confounds_timeseries.tsv'), sep='\t')

        # motion covars
        mc = confounds[['trans_x','trans_y','trans_z','rot_x', 'rot_y', 'rot_z']]
        mc_cov = make_motion_covariates(mc, tr)

        # events - NOTE: load bids events function does not work for other datasets not TOMOVA
        onsets = load_bids_events(subject=subject, run=run, tr=tr, task=task)

        # spikes
        spikes = data.find_spikes(global_spike_cutoff=3, diff_spike_cutoff=3)

        # Convolve, add linear drift, dct, intercept
        dm_cov = onsets.convolve().add_dct_basis(duration=120).add_poly(order=1, include_lower=True)

        # add spikes
        dm_cov = dm_cov.append(mc_cov, axis=1).append(Design_Matrix(spikes.iloc[:, 1:], sampling_freq=1/tr), axis=1)

        #regress
        data.X = dm_cov
        stats = data.regress()

        # Write out files
        for i, condition in enumerate([x for x in data.X.columns if 'Gain' in x]):
            stats['beta'][i].write(os.path.join(data_dir, 'derivatives', 'L1_Betas', f'sub-{subject:02}_task-{task}_run-{run:02}_{condition[:-3]}.nii.gz'))

## Aggregate Runs

In [ ]:
conditions = list(set([os.path.basename(x).split('_')[3] for x in glob.glob(os.path.join(data_dir, 'derivatives', 'L1_Betas', f'sub-*_task-flavor_run-*_*.nii.gz'))]))

all_data = {x:[] for x in conditions}
for subject in tqdm(range(1,16)):
    for condition in conditions:
        all_data[condition].append(Brain_Data(glob.glob(os.path.join(data_dir, 'derivatives', 'L1_Betas', f'sub-{subject:02}_task-flavor_run-*_{condition}_c0.nii.gz'))).mean())

for condition in list(all_data.keys()):
    Brain_Data(all_data[condition]).write(os.path.join(data_dir, 'derivatives', 'L1_Betas', f'Sweet_Taste_{condition}.hdf5'))

## Regression

In [ ]:
analysis_name = 'demeaned_zscore1_smooth0'
data_dir = '/Storage/Projects/Reward_Model'

zscore = lambda x: (x - np.mean(x, axis=0)) / np.std(x, axis=0)
center = lambda x: (x - np.mean(x, axis=0))

loss_aversion_metadata = pd.read_csv(os.path.join(data_dir, 'Analyses', 'Classification', f'Loss_Aversion_Similarity_{analysis_name}.csv'))

loss_aversion_metadata['Gain_z'] = loss_aversion_metadata['Gain'].groupby(loss_aversion_metadata['Subject']).transform(zscore)
loss_aversion_metadata['Loss_z'] = loss_aversion_metadata['Loss'].groupby(loss_aversion_metadata['Subject']).transform(zscore)

model = Lmer("Similarity ~ Gain + Loss + (Gain+Loss|Subject)", data=loss_aversion_metadata)
model.fit()



# Clean Fareri Social Dataset

In [ ]:
con_list = ['CompPun','CompRew', 'FriendPun','FriendRew','StrangerPun','StrangerRew']

sub_list = [os.path.basename(x) for x in glob.glob(os.path.join(data_dir, 'Data','Fareri_2012','Feedback_Betas','*'))]
sub_list.sort()


con_dat = {}
for con in con_list:
    con_dat[con] = []
    for sub in tqdm(sub_list):
        dat = Brain_Data(glob.glob(os.path.join(data_dir, 'Data','Fareri_2012','Feedback_Betas',sub, '*', f'{sub}*{con}.nii.gz')))
        con_dat[con].append(dat[dat.std(axis=1) > 0].mean())
    Brain_Data(con_dat[con]).write(os.path.join(data_dir, 'Data','Fareri_2012','Feedback_Summary', f'{con}.hdf5'))


# Clean Fareri Trust Dataset
Notes:
- Computer conditions have 2 subjects with empty regressors, we are excluding them only in the computer condition (n=24)
- All other conditions have 26 subjects, runs with empty regressors are excluded when calculating mean.

In [ ]:
con_list = ['ComputerReciprocateFB','ComputerDefectFB', 'StrangerReciprocateFB','StrangerDefectFB','FriendReciprocateFB','FriendDefectFB']

sub_list = [os.path.basename(x) for x in glob.glob(os.path.join(data_dir, 'Data','Fareri_2015','Feedback_Betas','*'))]
sub_list.sort()

con_dat = {}
for con in con_list:
    con_dat[con] = []
    for sub in tqdm(sub_list):
        dat = Brain_Data(glob.glob(os.path.join(data_dir, 'Data','Fareri_2015','Feedback_Betas',sub,f'{sub}*{con}.nii.gz')))
        con_dat[con].append(dat[dat.std(axis=1) > 0].mean())
# Brain_Data(con_dat[con]).write(os.path.join(data_dir, 'Data','Fareri_2015','Feedback_Summary', f'{con}.hdf5'))

# Check if data are empty
con_bool = {}
for con in con_list:
    con_bool[con] = [isinstance(x, Brain_Data) for x in con_dat[con]]

computer_sub_list = list(set(np.array(sub_list)[con_bool['ComputerReciprocateFB']]) & set(np.array(sub_list)[con_bool['ComputerDefectFB']]))
computer_sub_list.sort()

# Write out computer data
Brain_Data([x for x in con_dat['ComputerReciprocateFB'] if isinstance(x, Brain_Data)]).write(os.path.join(data_dir, 'Data','Fareri_2015','Feedback_Summary', f'ComputerReciprocateFB.hdf5'))
Brain_Data([x for x in con_dat['ComputerDefectFB'] if isinstance(x, Brain_Data)]).write(os.path.join(data_dir, 'Data','Fareri_2015','Feedback_Summary', f'ComputerDefectFB.hdf5'))

# Write out all the other conditions
for con in ['StrangerReciprocateFB','StrangerDefectFB','FriendReciprocateFB','FriendDefectFB']:
    Brain_Data(con_dat[con]).write(os.path.join(data_dir, 'Data','Fareri_2015','Feedback_Summary', f'{con}.hdf5'))
    Brain_Data(con_dat[con])[con_bool['ComputerReciprocateFB']].write(os.path.join(data_dir, 'Data','Fareri_2015','Feedback_Summary', f'{con}_n24.hdf5'))



# Milkshake Data
Notes about Dataset:

Stice, Burger, Yokum, 2015 J Neuro 

- 5 = Milkshake delivery
- 7 = Water delivery

**Contrasts**
 - con1 = milk cue - All Sessions
 - con2 = h20 cue - All Sessions'
 - con3 = milk receipt - All Sessions
 - con4 = h20 receipt - All Sessions
 - con5 = milk cue > h2o cue - All Sessions
 - con6 = h20 cue > milkshake cue - All Sessions
 - con7 = milk receipt > h20 receipt - All Sessions
 - con8 = h20 receipt > milkshake receipt - All Sessions
 - con9 = (milk cue & milk receipt ) > (h2o cue & h2o receipt) - All Sessions
 - con10 = (h2o cue & h2o receipt) > (milk cue & milk receipt ) - All Sessions
 
 
Sonja says that beta_0005 is milkshake receipt and beta_0007 is tasteless receipt